# 🤖 05_Final_Model_Training — Фінальне донавчання моделі
# Project: Прогнозування Відтоку Клієнтів
# Team Lead: Vitalii Subbotin
# Date: 2025-10-15

In [6]:
# 1. Імпорт бібліотек
import os
import joblib
import pandas as pd
import numpy as np
from pathlib import Path
from lightgbm import LGBMClassifier
from sklearn.preprocessing import StandardScaler


In [7]:
# 2. Налаштування шляхів
BASE_DIR = Path("..")
DATA_PATH = BASE_DIR / "data" / "df1_mean.csv"
MODELS_DIR = BASE_DIR / "models"
MODELS_DIR.mkdir(parents=True, exist_ok=True)

print(f"DATA_PATH -> {DATA_PATH}")
print(f"MODELS_DIR -> {MODELS_DIR}")


DATA_PATH -> ..\data\df1_mean.csv
MODELS_DIR -> ..\models


In [8]:
# 3. Завантаження даних
df = pd.read_csv(DATA_PATH)
print("Розмір датасету:", df.shape)

if 'id' in df.columns:
    df.drop(columns=['id'], inplace=True)
if 'Churn' in df.columns and 'churn' not in df.columns:
    df.rename(columns={'Churn': 'churn'}, inplace=True)

X = df.drop(columns=['churn'])
y = df['churn']

Розмір датасету: (71893, 12)


In [9]:
# 4. Масштабування числових ознак
binary_cols = [c for c in X.columns if X[c].nunique() <= 2]
num_cols = X.select_dtypes(include=[np.number]).columns.tolist()
cols_to_scale = [c for c in num_cols if c not in binary_cols]

scaler = StandardScaler()
X_scaled = X.copy()
X_scaled[cols_to_scale] = scaler.fit_transform(X[cols_to_scale])

scaler_path = MODELS_DIR / "scaler.pkl"
joblib.dump(scaler, scaler_path)
print(f"✅ Scaler збережено у: {scaler_path}")

✅ Scaler збережено у: ..\models\scaler.pkl


In [10]:
# 5. Навчання фінальної моделі LightGBM
best_params = {
    "learning_rate": 0.05,
    "max_depth": -1,
    "n_estimators": 300,
    "num_leaves": 63,
    "subsample": 0.8,
    "random_state": 42,
    "force_row_wise": True,
    "verbose": -1
}

model = LGBMClassifier(**best_params)
model.fit(X_scaled, y)

model_path = MODELS_DIR / "best_model_lgb.pkl"
joblib.dump(model, model_path)
print(f"✅ Модель збережено у: {model_path}")

✅ Модель збережено у: ..\models\best_model_lgb.pkl


In [11]:
# 6. Перевірка прогнозу
sample_pred = model.predict(X_scaled[:10])
sample_proba = model.predict_proba(X_scaled[:10])[:, 1]
print("\nПриклад прогнозів:", sample_pred)
print("Ймовірності:", np.round(sample_proba, 3))

print("\n📌 Фінальна модель LightGBM навчена на всьому датасеті та збережена.")
print(f"Артефакти:\n - {model_path}\n - {scaler_path}")


Приклад прогнозів: [0 1 1 1 1 0 1 1 1 1]
Ймовірності: [0.006 0.997 0.998 0.967 0.999 0.073 0.999 0.999 0.997 0.997]

📌 Фінальна модель LightGBM навчена на всьому датасеті та збережена.
Артефакти:
 - ..\models\best_model_lgb.pkl
 - ..\models\scaler.pkl
